In [ ]:
import requests, json, re, datetime, dateutil.parser

In [ ]:
token = 'your token'

In [ ]:
response = requests.get('https://graph.microsoft.com/beta/me/joinedTeams', headers={'Authorization': token})
if (response.status_code == 200):
    print('Success')
else:
    print('Error: ' + str(response))


In [ ]:
teams = json.loads(response.content)
for team in teams['value']:
    print(team['id'] + '   ' + team['displayName'])

In [ ]:
# Input the a groupID UUID from previous step
groupID = "3055aa2d-b68d-4a8d-861c-0965de9ad15d"

# Get list of channels
response = requests.get('https://graph.microsoft.com/beta/teams/3055aa2d-b68d-4a8d-861c-0965de9ad15d/channels', headers={'Authorization': token})
if (response.status_code == 200):
    channels = json.loads(response.content)
    for channel in channels['value']:
        print(channel['id'] + '   ' + channel['displayName'])
else:
    print('Error: ' + str(response))



In [ ]:
# Do all for channels for single team

groupID = "3055aa2d-b68d-4a8d-861c-0965de9ad15d"

for channel in channels['value']:
    print(channel['id'] + '   ' + channel['displayName'])

    # Get JSON and save to file
    chat = pullMessagesIntoJSON(groupID, channel['id'])
    f = open('output/' + channel['displayName'] + "_output.json", "w")
    f.write(json.dumps(chat))
    f.close()

    # Parse into HTML and save to file
    chatHTML = parseJSONintoHTML(chat)
    f = open('output/' + channel['displayName'] + "_output.html", "w")
    f.write(json.dumps(chatHTML))
    f.close()

print('All done!')

In [ ]:
def pullfromAPI(url):
    # print(url)
    response = requests.get(url, headers={'Authorization': token})
    messages = json.loads(response.content)
    # print("Pulled " +  " items.")
    return messages

def pullMessagesIntoJSON(_groupID, _channelID):
    # Gather all messages
    allMessagesRaw = []
    linkToNextBatch = ""

    # Get list of channels
    messages = pullfromAPI('https://graph.microsoft.com/beta/teams/' + _groupID + '/channels/' + _channelID + '/messages?$top=100')
    for item in messages["value"]: allMessagesRaw.append(item)

    # If there's another batch
    if "@odata.nextLink" in messages.keys():
        linkToNextBatch = messages["@odata.nextLink"]
        print('Another batch available')

        while True:
            messages = pullfromAPI(linkToNextBatch)
            for item in messages["value"]: allMessagesRaw.append(item)

            if "@odata.nextLink" in messages.keys():
                linkToNextBatch = messages["@odata.nextLink"]
                print('Another batch available')
            else:
                break

    print('Done with pulling messages! Now adding in replies...')

    # For each message, pull replies and add to dict
    for msg in allMessagesRaw: 
        replies = pullfromAPI('https://graph.microsoft.com/beta/teams/' + _groupID + '/channels/' + _channelID + '/messages/' + msg["id"] + '/replies')
        msg["replies"] = []

        if replies['@odata.count'] > 0:
            for reply in replies["value"]: msg["replies"].append(reply)

            # Check if more
            if "@odata.nextLink" in replies.keys():
                while True:
                    linkToNextBatch = replies["@odata.nextLink"]
                    print('Another batch available')
                    replies = pullfromAPI(linkToNextBatch)
                    for reply in replies["value"]: msg["replies"].append(reply)

                    # If more replies, repeat, otherwise break
                    if "@odata.nextLink" in replies.keys():
                        linkToNextBatch = replies["@odata.nextLink"]
                        print('Another batch available')
                    else:
                        break
    print('Collected ' + str(len(allMessagesRaw)) + ' messages!')
    return allMessagesRaw

def parseJSONintoHTML(jsonChatMessages):
    finalHTMLOutput = ""

    jsonChatMessages.sort(key=lambda x: dateutil.parser.isoparse(x['createdDateTime']))

    for msg in jsonChatMessages: 
        b = msg['body']['content']
        if b is not None:
            b = re.sub('\n+', '', b)
            b = re.sub('\t+', '', b)

            finalHTMLOutput += '<hr><hr><h3>' + msg['from']['user']['displayName'] + ':</h3><h5>Created: ' + msg["createdDateTime"] + '</h5>' + b + '<blockquote>'

        msg['replies'].sort(key=lambda x: dateutil.parser.isoparse(x['createdDateTime']))
        for reply in msg['replies']:
            try:
                user = str(reply['from']['user']['displayName'])
            except TypeError:
                user = 'unknown'
                pass

            try:
                replyContent = reply['body']['content']
                replyContent = re.sub('\n+', '', replyContent)
                replyContent = re.sub('\t+', '', replyContent)
            except TypeError:
                replyContent = 'unknown'
                pass

            finalHTMLOutput += '<h3>Reply From: ' + user + '</h3>' + '<h5>Created: ' + reply["createdDateTime"] + '</h5>' + replyContent

        finalHTMLOutput += '</blockquote>'
    return finalHTMLOutput


In [ ]:


f = open("Crowdsourcing.html", "w")
f.write(json.dumps(finalHTMLOutput))
f.close()

print('done')

In [ ]:
# Users
allUsers = []
f = open('users_output.json', "w")

messages = pullfromAPI('https://graph.microsoft.com/v1.0/users')
for item in messages["value"]: f.write(json.dumps(item))


# If there's another batch
if "@odata.nextLink" in messages.keys():
    linkToNextBatch = messages["@odata.nextLink"]
    i = 0

    while True:
        messages = pullfromAPI(linkToNextBatch)
        for item in messages["value"]: f.write(json.dumps(item))

        if "@odata.nextLink" in messages.keys():
            linkToNextBatch = messages["@odata.nextLink"]
            # print('Another batch available')
        else:
            break

print('Done!')